<img src=https://api-manager.universia.net/coreplatform-document-management/api/document-management/public/6ra7ymjkfi64845 width="300" align="left">
<br />

# Aprendizaje automático I
#### <font color=green>*Máster en Informática Industrial y Robótica*</font>

Daniel Vidal Soroa  
Juan Diego Peña 

## Clasificación de mangos en tres clases según su presencia para la exportación, comercio local o procesamiento industrial

#### Importación de las librerías:

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.utils import shuffle
from skimage.io import imread
from skimage.transform import resize
from sklearn.metrics import roc_curve, auc, plot_confusion_matrix, classification_report, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split
from random import seed
from random import randint
from tensorflow import keras
from keras import layers, models, utils, datasets
from keras import utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold, GridSearchCV

In [2]:
%store -r scoring X X_train X_val CV t t_val f n 


#### Validación cruzada con LDA

In [6]:
modelLDA = LinearDiscriminantAnalysis()

scoresLDA = cross_validate(modelLDA, X, t, cv=CV, scoring=scoring,n_jobs=-1)

print('Media del LDA accuracy : ', np.mean(scoresLDA['test_accuracy']))

In [4]:
df = pd.DataFrame.from_dict(scoresLDA)
df.drop(df.iloc[:, 0:2], inplace = True, axis = 1)
df.loc[5]=[np.mean(scoresLDA['test_recall_macro']),np.mean(scoresLDA['test_precision_macro']),np.mean(scoresLDA['test_f1_macro']),np.mean(scoresLDA['test_accuracy']),np.mean(scoresLDA['test_roc_auc_ovo'])]
df.index = ["CV 1","CV 2","CV 3","CV 4","CV 5","Average"]
df.to_excel("./informe/tables/LDA.xlsx") 
print(df)


         test_recall_macro  test_precision_macro  test_f1_macro  \
CV 1              0.940741              0.941477       0.940838   
CV 2              0.937037              0.937085       0.937023   
CV 3              0.932407              0.932407       0.932407   
CV 4              0.950926              0.951117       0.950884   
CV 5              0.940741              0.941034       0.940597   
Average           0.940370              0.940624       0.940350   

         test_accuracy  test_roc_auc_ovo  
CV 1          0.940741          0.972488  
CV 2          0.937037          0.972411  
CV 3          0.932407          0.976434  
CV 4          0.950926          0.977809  
CV 5          0.940741          0.975062  
Average       0.940370          0.974841  


### Analisis hiper-parametros LDA  

In [5]:
param_grid = {
    "solvers": ['svd', 'lsqr'] 
}
search = GridSearchCV(estimator = modelLDA, param_grid =param_grid, n_jobs=-1,cv=2,scoring=scoring,verbose=5,return_train_score=False,refit=False)

search_results = search.fit(X,t)



Fitting 2 folds for each of 3 candidates, totalling 6 fits


In [ ]:
df = pd.DataFrame.from_dict(search_results.cv_results_)
#df.to_excel("./informe/tables/parameters_knn.xlsx") 
labels = df['params']

data_recall =[]
data_precision = []
data_f1 =[]
data_accuracy = []
data_roc_auc  =[]


for c in range(len(labels)): 
    data_recall.append(df.iloc[c,8:13].to_list())  
    data_precision.append(df.iloc[c,16:21].to_list())  
    data_f1.append(df.iloc[c,24:29].to_list())  
    data_accuracy.append(df.iloc[c,32:37].to_list())  
    data_roc_auc.append(df.iloc[c,40:45].to_list())    





fig, ax = plt.subplots(5,1)

ax[0].boxplot(data_recall,labels=labels)
ax[0].set_xticklabels(labels=labels,rotation=30, fontsize=8)
ax[0].set(
    axisbelow=True,  # Hide the grid behind plot objects
    title='Comparación de hiperaparametros',
    ylabel='Recall'
    )

ax[1].boxplot(data_precision,labels=labels)
ax[1].set_xticklabels(labels=labels,rotation=30, fontsize=8)
ax[1].set(
    axisbelow=True,  # Hide the grid behind plot objects
    ylabel='Precision'
)

ax[2].boxplot(data_f1,labels=labels)
ax[2].set_xticklabels(labels=labels,rotation=30, fontsize=8)
ax[2].set(
    axisbelow=True,  # Hide the grid behind plot objects
    ylabel='f1'
)


ax[3].boxplot(data_accuracy,labels=labels)
ax[3].set_xticklabels(labels=labels,rotation=30, fontsize=8)
ax[3].set(
    axisbelow=True,  # Hide the grid behind plot objects
    ylabel='accuracy'
)


ax[4].boxplot(data_roc_auc,labels=labels)
ax[4].set_xticklabels(labels=labels,rotation=30, fontsize=8)
ax[4].set(
    axisbelow=True,  # Hide the grid behind plot objects
    ylabel='roc_auc'
)
fig.subplots_adjust(left=0.08, right=3, bottom=0.05, top=9,
                    hspace=0.4, wspace=0.3)

plt.show()
